# Geoqueries en Mongo

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#¿Qué-podemos-hacer-con-las-geoqueries?" data-toc-modified-id="¿Qué-podemos-hacer-con-las-geoqueries?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>¿Qué podemos hacer con las geoqueries?</a></span></li><li><span><a href="#Generamos-la-conexión-con-Mongo" data-toc-modified-id="Generamos-la-conexión-con-Mongo-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generamos la conexión con Mongo</a></span></li><li><span><a href="#Importar-colecciones" data-toc-modified-id="Importar-colecciones-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Importar colecciones</a></span></li><li><span><a href="#Crear-indexes" data-toc-modified-id="Crear-indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Crear indexes</a></span></li><li><span><a href="#Generamos-las-conexiones-con-la-base-de-datos" data-toc-modified-id="Generamos-las-conexiones-con-la-base-de-datos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generamos las conexiones con la base de datos</a></span></li><li><span><a href="#Empezamos-con-las-Geoqueries" data-toc-modified-id="Empezamos-con-las-Geoqueries-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Empezamos con las Geoqueries</a></span></li><li><span><a href="#Intersects" data-toc-modified-id="Intersects-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Intersects</a></span></li><li><span><a href="#GeoWithin" data-toc-modified-id="GeoWithin-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>GeoWithin</a></span></li><li><span><a href="#Near" data-toc-modified-id="Near-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Near</a></span></li><li><span><a href="#Extra,-extra!!-mis-datos-extraídos-de-Foursquare-en-Mongo" data-toc-modified-id="Extra,-extra!!-mis-datos-extraídos-de-Foursquare-en-Mongo-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Extra, extra!! mis datos extraídos de Foursquare en Mongo</a></span><ul class="toc-item"><li><span><a href="#Voy-a-traerme-todos-los-restaurantes-veganos-de-Madrid" data-toc-modified-id="Voy-a-traerme-todos-los-restaurantes-veganos-de-Madrid-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Voy a traerme todos los restaurantes veganos de Madrid</a></span></li></ul></li><li><span><a href="#Vamos-a-ver-si-hay-alguno-cerca-del-matadero-/-también-las-fruterías" data-toc-modified-id="Vamos-a-ver-si-hay-alguno-cerca-del-matadero-/-también-las-fruterías-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Vamos a ver si hay alguno cerca del matadero / también las fruterías</a></span><ul class="toc-item"><li><span><a href="#Los-veganos" data-toc-modified-id="Los-veganos-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Los veganos</a></span></li></ul></li></ul></div>

## ¿Qué podemos hacer con las geoqueries?
Con las geoquerías podemos responder a preguntas como las siguientes
 * ¿Dónde están las farmacias más cercanas a mi ubicación?
 * ¿Qué restaurantes hay en esta poligonal?

Cada punto de la Tierra se caracteriza por dos números:
 * Longitud: ángulo respecto al meridiano de Greenwich. Va de -180° (antiGreenwich) a +180° (también antiGreenwich)
 * Latitud: ángulo con respecto al ecuador. Va de -90° (sur) a +90° (norte)

![latierra](../images/latlon.gif)

## Generamos la conexión con Mongo

In [3]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

## Importar colecciones     
Sigamos la documentación oficial de Mongo: [Mongo Geo Example](https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/)
Creamos estas dos colecciones nuevas:
 * `restaurants2`
 * `neighborhoods`

## Crear indexes
La indexación geoespacial de MongoDB permite ejecutar eficientemente **consultas espaciales** sobre una colección que contiene formas y puntos geoespaciales.
Vamos a crear un índice geográfico para que Mongo sepa que queremos hacer geo-consultas en esta colección.  
Esto sólo debe hacerse una vez, y ahora la colección de Mongo está correctamente indexada para siempre.

In [5]:
from pymongo import GEOSPHERE

In [8]:
# Creamos el  index 2dsphere desde pymongo
db.barrios.create_index([("geometry", GEOSPHERE)])

'geometry_2dsphere'

Es importante indicarle el campo donde tenemos tanto el tipo de elemento (polígonos o  puntos (point)) como las coordenadas.

In [10]:
# Si quiero crear el de restaurants
db.restaurants2.create_index([("location", GEOSPHERE)])

## Generamos las conexiones con la base de datos

In [11]:
r = db.get_collection("restaurants2")
b = db.get_collection("barrios")

## Empezamos con las Geoqueries   
¿En qué barrio estoy?      
¿Qué elementos **intersecan** el elemento dado?

## Intersects

Necesitamos tener los datos  con tipo polígono en este caso tenemos una colección con barrios donde cada barrio  tiene dentro  el polígono que lo delimita. Y cada barrio es un documento, por eso cuando  intersecta me devuelve el documento y yo veo  que es el barrio  en concreto.    
Selecciona los documentos cuyos datos geoespaciales se cruzan con un objeto GeoJSON especificado; es decir, cuando la intersección de los datos y el objeto especificado no está vacía.

In [13]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista
    }

In [12]:
coordenadas = [-73.93, 40.82]

In [14]:
type_point(coordenadas)

{'type': 'Point', 'coordinates': [-73.93, 40.82]}

In [15]:
proj = {"_id": 0, "name": 1}

In [17]:
b.find_one().keys()

dict_keys(['_id', 'geometry', 'name'])

El geometry que va sin dolar es porque vamos a buscar en la colección de barrios y donde está la información es dentro de la key geometry, se ve justo aquí arriba 👆🏻. El geometry que lleva dolar es la sintaxis de mongo para todas las queries geoespaciales. 
Para hacer una query geoespacial a Mongo tengo que pasarle las coordenadas en tipo point

In [18]:
b.find_one({"geometry": {"$geoIntersects": {"$geometry":type_point(coordenadas) }}}, proj)

{'name': 'West Concourse'}

In [23]:
# LO QUE LLAMÁBAMOS FILTRO A LA HORA DE HACER LA QUERY
filtro = {"geometry": {"$geoIntersects": {"$geometry":type_point(coordenadas)}}}

In [26]:
#list(b.find(filtro))

In [27]:
list(b.find(filtro, proj))

[{'name': 'West Concourse'}]

## GeoWithin

¿Qué elementos están **contenidos** en el elemento dado?     
¿Qué restaurantes hay en este barrio (polígono)?

In [28]:
# Vemos la estructura de los documentos sacando uno y viendo sus keys
b.find_one({}).keys()

dict_keys(['_id', 'geometry', 'name'])

In [29]:
#bUSCAMOS UN barrio en  concreto
barrio = b.find_one({"name": "West Concourse"})

In [32]:
#Extraigo las coordenadas
coordenadas = barrio["geometry"]

In [33]:
#Saco las keys de un documento de la coleccción  de restaurantes
r.find_one({}).keys()

dict_keys(['_id', 'location', 'name'])

In [ ]:
# ER FIRTRO
#Location porque la key de la colección donde están las coordenadas se llama location

In [41]:
query = r.find({"location": {"$geoWithin": {"$geometry": coordenadas}}})

In [42]:
restaurantes_west = list(query)

In [43]:
len(restaurantes_west)

125

In [46]:
import pandas as pd
df = pd.DataFrame(restaurantes_west)

In [47]:
df.head()

,_id,location,name
0,55cba2476c522cafdb056004,"{'coordinates': [-73.9259201, 40.8278293], 'ty...",Hard Rock Cafe
1,55cba2476c522cafdb056005,"{'coordinates': [-73.9259201, 40.8278293], 'ty...",Nyy Steak
2,55cba2476c522cafdb058ae9,"{'coordinates': [-73.9259245, 40.827435], 'typ...",Dunkin Donuts
3,55cba2476c522cafdb0543cb,"{'coordinates': [-73.92594439999999, 40.827212...",Billy'S Sport Bar Restaurant & Lounge
4,55cba2476c522cafdb056784,"{'coordinates': [-73.9262845, 40.8266956999999...",Stan'S Sports Bar


## Near

¿Qué elementos se encuentran a una distancia de una geometría determinada?     
`$nearSphere`/`$near` y `$maxDistance` nos ayudan a encontrar todos los elementos dentro de `maxDistance` metros de la geometría deseada, **ordenados** de más cercano a más lejano.

In [48]:
#  Mi posición, voy a buscar restaurantes cercanos a  ESTE punto
coordenadas2 = [-73.93, 40.82]

In [53]:
# Km que quiero  caminar
metros = 300

In [54]:
coord = type_point(coordenadas2)

In [55]:
query = {"location": {"$near": {"$geometry": coord,"$minDistance":100 ,"$maxDistance": metros}}}

In [57]:
list(r.find(query))[0]

{'_id': ObjectId('55cba2476c522cafdb05517e'),
 'location': {'coordinates': [-73.92856720000002, 40.8190635],
  'type': 'Point'},
 'name': 'Glacken Bar'}

In [64]:
res = pd.DataFrame(list(r.find(query)))
res.head()

,_id,location,name
0,55cba2476c522cafdb05517e,"{'coordinates': [-73.92856720000002, 40.819063...",Glacken Bar
1,55cba2476c522cafdb055dae,"{'coordinates': [-73.9284026, 40.8190186], 'ty...",Sunlight Restaurant
2,55cba2476c522cafdb055ab1,"{'coordinates': [-73.9280411, 40.8189109], 'ty...",The News Room Jazz Club
3,55cba2476c522cafdb054247,"{'coordinates': [-73.92789290000002, 40.818967...",Glen Roy Bar And Grill
4,55cba2486c522cafdb059dad,"{'coordinates': [-73.92778, 40.819083], 'type'...",


In [61]:
barrio_cerca = {"geometry": {"$near": {"$geometry": coord,"$minDistance":100 ,"$maxDistance": 300}}}

In [62]:
list(b.find(barrio_cerca, proj))

[{'name': 'East Concourse-Concourse Village'},
 {'name': 'Mott Haven-Port Morris'}]

## Extra, extra!! mis datos extraídos de Foursquare en Mongo

### Voy a traerme todos los restaurantes veganos de Madrid

In [101]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

True

In [194]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {
            "type": "Point",
            "coordinates": [float(data["latt"]), float(data["longt"])]
        }
    
    except:
        return data

In [106]:
dondebuscamos = geocode("Madrid")
dondebuscamos

{'type': 'Point', 'coordinates': ['40.42955', '-3.67930']}

In [115]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [116]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [117]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{dondebuscamos['coordinates'][0]}, {dondebuscamos['coordinates'][1]}",
    "query": "Vegan",
    "limit": 200   
}

In [120]:
resp = requests.get(url_query, params=parametros).json()["response"]["groups"][0]["items"]

In [124]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]
mapa_direccion = ["venue", "location", "formattedAddress"]

In [128]:
def getFromDict(diccionario, mapa):
    from functools import reduce
    import operator
    return reduce(operator.getitem,mapa,diccionario)

In [129]:
unjsonnuevo = []
for dic in resp:
    paralista= {}
    paralista["nombre"]= getFromDict(dic,mapa_nombre)
    paralista["latitud"]= getFromDict(dic,mapa_latitud)
    paralista["longitud"]= getFromDict(dic,mapa_longitud)
    paralista["dirección"]= getFromDict(dic,mapa_direccion)
    paralista["location"]=  type_point([paralista["latitud"],paralista["longitud"]])
    unjsonnuevo.append(paralista)

In [130]:
unjsonnuevo[0]

{'nombre': 'Planet Organic Food&Life',
 'latitud': 40.428808,
 'longitud': -3.681333,
 'dirección': ['63 Calle Castelló', '28001 Madrid Madrid', 'España'],
 'location': {'type': 'Point', 'coordinates': [40.428808, -3.681333]}}

In [139]:
df = pd.DataFrame(unjsonnuevo)
df.sample()

,nombre,latitud,longitud,dirección,location
2,Bunny's Deli,40.423935,-3.697158,"[Calle San Gregorio 17, 28004 Madrid Madrid, E...","{'type': 'Point', 'coordinates': [40.423935, -..."


In [138]:
# Voy a hacer un geodataframe por si quiero utilizar carto

In [142]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitud, df.latitud))
gdf.sample()

,nombre,latitud,longitud,dirección,location,geometry
7,Honest Greens,40.428914,-3.687266,"[28001 Madrid Madrid, España]","{'type': 'Point', 'coordinates': [40.428914, -...",POINT (-3.68727 40.42891)


In [171]:
import json
with open('vegan.json', 'w') as f:
    json.dump(unjsonnuevo,f)

Los importo a una colección de Mongo llamada vegan

## Vamos a ver si hay alguno cerca del matadero / también las fruterías

In [65]:
frut = db.get_collection("frut")

In [68]:
frut.find_one({})

{'_id': ObjectId('6130ab0bf57cd4cd4b93554e'),
 'nombre': 'Platea Madrid',
 'latitud': 40.426083288964996,
 'longitud': -3.68866416486341,
 'location': {'type': 'Point',
  'coordinates': [40.426083288964996, -3.68866416486341]}}

In [90]:
coord = [40.3930059,-3.7010902]
lavapi = [40.408808, -3.7034796]

In [100]:
type_point

<function __main__.type_point(lista)>

In [97]:
query = {"location": {"$near":{"$geometry": type_point(lavapi), "$maxDistance": 900}}}

In [98]:
list(frut.find(query))

[{'_id': ObjectId('6130ab0bf57cd4cd4b935555'),
  'nombre': 'Fruteria Toledo',
  'latitud': 40.408692002017396,
  'longitud': -3.710762120782159,
  'location': {'type': 'Point',
   'coordinates': [40.408692002017396, -3.710762120782159]}}]

### Los veganos

In [178]:
veg = db.get_collection("vegan")

In [207]:
veg.find_one({})

{'_id': ObjectId('6130b2cbc5c899dec404edec'),
 'nombre': 'Planet Organic Food&Life',
 'latitud': 40.428808,
 'longitud': -3.681333,
 'dirección': ['63 Calle Castelló', '28001 Madrid Madrid', 'España'],
 'location': {'type': 'Point', 'coordinates': [40.428808, -3.681333]}}

Creamos el Index 2dsphere a través de MongoDB compass

In [197]:
ironhack = geocode("Matadero de Madrid")
ironhack

{'type': 'Point', 'coordinates': [40.46338, -3.70338]}

In [221]:
veganoscerca = {"location": {"$near":{"$geometry": ironhack, "$maxDistance": 4000}}}

In [222]:
list(veg.find(veganoscerca))

[{'_id': ObjectId('6130b2cbc5c899dec404edf6'),
  'nombre': 'Goiko Grill',
  'latitud': 40.437772891427834,
  'longitud': -3.686615310483156,
  'dirección': ['C. María de Molina, 20', '28006 Madrid Madrid', 'España'],
  'location': {'type': 'Point',
   'coordinates': [40.437772891427834, -3.686615310483156]}}]